In [1]:
import io
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
conn= sqlite3.connect('tp1.db')
cursor= conn.cursor()

In [ ]:
f= io.open('despesas_publicas_tp1.sql', 'r')
sql= f.read()
cursor.executescript(sql)

In [ ]:
df= pd.read_sql_query("SELECT * FROM ORGAO ORDER BY QTD_ORGAOS_SUBORDINADOS DESC", conn)
df

In [ ]:
df = pd.read_sql_query("SELECT P.NOME_PROGRAMA_GOVERNO, SUM(D.VALOR_DESPESA) AS VALOR_DESPESA " + 
                       "FROM DESPESA_PUBLICA AS D " +
                       "INNER JOIN PROGRAMA_GOVERNO AS P " +
                       "ON D.COD_PROGRAMA_GOVERNO = P.COD_PROGRAMA_GOVERNO " + 
                       "GROUP BY P.NOME_PROGRAMA_GOVERNO " +
                       "HAVING SUM(VALOR_DESPESA) > 0 " +
                       "ORDER BY VALOR_DESPESA DESC", conn)
df

In [ ]:
df = pd.read_sql_query("SELECT D.MES_DESPESA, SUM(D.VALOR_DESPESA) AS VALOR_DESPESA " + 
                       "FROM DESPESA_PUBLICA AS D " +
                       "INNER JOIN DETALHAMENTO_FUNCAO AS DF " +
                       "ON D.COD_DETALHAMENTO = DF.COD_DETALHAMENTO " +
                       "INNER JOIN FUNCAO AS F " +
                       "ON DF.COD_FUNCAO = F.COD_FUNCAO " +
                       "WHERE F.NOME_FUNCAO = 'Educação' " +
                       "GROUP BY D.MES_DESPESA " +
                       "ORDER BY D.MES_DESPESA", conn)
df

In [ ]:
plt.bar(df['MES_DESPESA'].tolist(), df['VALOR_DESPESA'].tolist())

plt.xlabel("Mês/2018")
plt.ylabel("Total Despesa Mensal")
plt.show()

In [ ]:
#(0+1)Liste todos os codigos e nomes de func~oes da subfunc~ao que possui o maior numero de func~oes.
df= pd.read_sql_query("SELECT F.cod_funcao, nome_funcao FROM Funcao F "+
                      "WHERE F.cod_funcao IN (SELECT D.cod_funcao, count(*) NroSub FROM Detalhamento_Funcao D "+
                          "GROUP BY D.cod_subfuncao)", conn)
df

In [22]:
#(9+1)Liste todas as despesas publicas (codigo da despesa, m^es e ano da despesa, modalidade,
#valor), ordenadas de forma decrescente pelo valor, em que o programa de governo e
#aquele que possui o maior valor total em despesas durante o ano de 2018.
df= pd.read_sql_query("SELECT cod_despesa, mes_despesa, ano_despesa, modalidade_despesa, valor_despesa FROM Despesa_Publica "+
                      "WHERE ano_despesa=2018 AND cod_programa_governo = "+
                          "(SELECT cod_programa_governo FROM Despesa_Publica WHERE valor_despesa="+
                              "(SELECT MAX(valor_despesa) FROM Despesa_Publica))"+
                      " ORDER BY valor_despesa DESC", conn)
df

,COD_DESPESA,MES_DESPESA,ANO_DESPESA,MODALIDADE_DESPESA,VALOR_DESPESA
0,835,9,2018,Reserva de Contingência,334126.70
1,300,6,2018,Reserva de Contingência,135.36
2,283,1,2018,APLICACOES DIRETAS - OPER.INTRA-ORCAMENTARIAS,0.00
3,284,2,2018,APLICACOES DIRETAS - OPER.INTRA-ORCAMENTARIAS,0.00
4,285,3,2018,APLICACOES DIRETAS - OPER.INTRA-ORCAMENTARIAS,0.00
5,286,4,2018,APLICACOES DIRETAS - OPER.INTRA-ORCAMENTARIAS,0.00
6,287,5,2018,APLICACOES DIRETAS - OPER.INTRA-ORCAMENTARIAS,0.00
7,288,6,2018,APLICACOES DIRETAS - OPER.INTRA-ORCAMENTARIAS,0.00
8,289,7,2018,APLICACOES DIRETAS - OPER.INTRA-ORCAMENTARIAS,0.00
9,290,8,2018,APLICACOES DIRETAS - OPER.INTRA-ORCAMENTARIAS,0.00


In [ ]:
#(8+1)Liste todas as despesas publicas (codigo da despesa, m^es e ano da despesa, modalidade,
#valor) em que o org~ao responsavel e aquele que possui um numero de org~aos
#subordinados maior do que a media.

In [12]:
#(6+1)Liste os programas de governo e a quantidade de org~aos governamentais distintos envolvidos
#nas despesas publicas desses programas. Nesta consulta, desconsidere valores
#de despesa negativos ou iguais a zero no perodo.
df= pd.read_sql_query("SELECT nome_programa_governo, count(cod_orgao) as NUMERO_ORGAOS, valor_despesa "+
                      "FROM Orgao NATURAL JOIN Despesa_Publica NATURAL JOIN Programa_Governo " +
                      "WHERE valor_despesa>0 GROUP BY cod_programa_governo "+
                      "ORDER BY nome_programa_governo", conn)
df

,NOME_PROGRAMA_GOVERNO,NUMERO_ORGAOS,VALOR_DESPESA
0,BRASIL SEM MISERIA,6,8215.70
1,CONTROLE DO CANCER,1,38971.19
2,ENEM,2,135.36
3,MAIS MEDICOS,3,2400.00
4,PRONATEC,7,5000.00
5,REDE CEGONHA,1,1210.00
6,REGULARIZACAO FUNDIARIA,1,1497.26


In [34]:
#(8+1+1+1)Liste os codigos e nomes dos org~aos governamentais relacionados ao programa de
#governo (ou programas, caso haja empate) que alcancou mais registros de despesas
#publicas durante o ano.
df= pd.read_sql_query("SELECT cod_orgao, nome_orgao FROM Orgao NATURAL JOIN Despesa_Publica WHERE cod_programa_governo IN "+
                          "(SELECT cod_programa_governo FROM (SELECT cod_programa_governo, max(NRO_REGISTROS) FROM "+
                              "(SELECT cod_programa_governo, count(cod_programa_governo) as NRO_REGISTROS "+
                                  "FROM Despesa_Publica GROUP BY cod_programa_governo)))",conn)
df

,COD_ORGAO,NOME_ORGAO
0,20000,Presidência da República
1,20000,Presidência da República
2,20000,Presidência da República
3,20000,Presidência da República
4,20000,Presidência da República
5,20000,Presidência da República
6,20000,Presidência da República
7,20000,Presidência da República
8,22000,"Ministério da Agricultura, Pecuária e Abastec"
9,22000,"Ministério da Agricultura, Pecuária e Abastec"


In [32]:
#AUXILIAR PARA 11
df= pd.read_sql_query("SELECT cod_programa_governo, count(cod_programa_governo) as NRO_REGISTROS FROM Despesa_Publica "+
                                  "GROUP BY cod_programa_governo",conn)
df

,COD_PROGRAMA_GOVERNO,NRO_REGISTROS
0,1,150
1,2,3
2,3,16
3,4,9
4,5,12
5,6,38
6,7,40
7,8,24
8,9,1
9,11,53


In [ ]:
#(8+1+1+1+1)Liste a media de despesas publicas por func~ao (nome) e subfunc~ao (nome), apenas nos
#casos em que a media excedeu mil reais.
df= pd.read_sql_query("SELECT nome_funcao, nome_subfuncao, avg(valor_despesa) AS MEDIA_DESPESAS "+
                      "FROM (Subfuncao NATURAL JOIN Detalhamento_Funcao NATURAL JOIN Funcao) "+
                      "NATURAL JOIN Despesa_Publica GROUP BY cod_funcao, cod_subfuncao HAVING avg(valor_despesa)>1000 "+
                      "ORDER BY avg(valor_despesa) DESC", conn)
df

In [ ]:
df= pd.read_sql_query("SELECT DISTINCT nome_funcao, valor_despesa "+
                      "FROM Funcao NATURAL JOIN Detalhamento_Funcao NATURAL JOIN Despesa_Publica", conn)
df

In [ ]:
df= pd.read_sql_query("SELECT DISTINCT modalidade_despesa, valor_despesa, nome_orgao "+
                      "FROM Orgao NATURAL JOIN (SELECT * FROM Despesa_Publica "+
                      "WHERE modalidade_despesa= 'Reserva de Contingência')", conn)
df

In [ ]:
df= pd.read_sql_query("SELECT DISTINCT valor_despesa, nome_programa_governo "+
                      "FROM Despesa_Publica NATURAL JOIN (SELECT * FROM Programa_Governo "+
                      "WHERE nome_programa_governo='ENEM')", conn)
df

In [ ]:
df= pd.read_sql_query("SELECT DISTINCT nome_programa_governo, qtd_orgaos_subordinados, valor_despesa "+
                      "FROM Despesa_Publica NATURAL JOIN (SELECT * FROM Orgao "+
                      "WHERE nome_orgao= 'Ministério da Educação') NATURAL JOIN Programa_Governo", conn)
df

In [ ]:
conn.close()